<a href="https://colab.research.google.com/github/sofaemha/pengolahan-citra-digital/blob/main/26-Maret-2024/klasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tugas PCD: Klasifkasi Gambar

Dengan dibuatnya tugas ini bertujuan untuk membangun sebuah model neural network yang dapat mengklasifikasikan gambar angka, seperti nol dan delapan. Tugas ini menggunakan fitur [Keras](https://www.tensorflow.org/guide/keras), sebuah API tingkat tinggi untuk membangun dan melakukan training model di TensorFlow.

## Persiapan dan Konfigurasi

Saat ini TensorFlow berada pada versi `2.x`, sebelum melakukan impor data alangkah baiknya menguji apakah versi aplikasi tersedia, menggunakan kode berikut :

In [18]:
try:
  # Inisialisasi versi dari TensorFlow
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Jika proses inisialisasi berhasil, selanjutnya adalah melakukan impor data dan fungsi yang diperlukan dengan mengeksekusi kode berikut :

In [19]:
# Pustaka utama
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow dan API Keras
import tensorflow
from tensorflow import keras

# Pustaka penunjang
import numpy as numerik
import matplotlib.pyplot as matlab

# Mengecek versi TensorFlow
print(tensorflow.__version__)

2.15.0


## Konfigurasi Dataset

Tugas ini menggunakan dataset [MNIST Digits](https://keras.io/api/datasets/mnist/) yang memiliki 60,000 gambar hitam putih dalam 10 kategori angka. Tiap gambar menunjukan gambar dari satu tipe angka dalam resolusi rendah (28 x 28 pixel), seperti yang dapat dilihat disini :

<table>
  <tr><td>
    <img src="https://www.researchgate.net/profile/Bin-Dai-9/publication/331768880/figure/fig3/AS:736574090260480@1552624356565/MNIST-samples-generated-by-2-stage-VAE-no-cherry-picking.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Gambar 1.</b> <a href="https://keras.io/api/datasets/mnist/">MNIST samples generated by 2-stage VAE</a> (by Bin Dai).<br/>&nbsp;
  </td></tr>
</table>

Dataset MNIST terdiri atas gambar angka tulisan tangan (0, 1, 2, dsb) dalam format yang identik dengan gambar angka yang akan digunakan dalam dataset MNIST.

Terdapat 60,000 gambar digunakan sebagai data train dari model neural network dan 10,000 gambar digunakan untuk mengevaluasi seberapa akurat model dapat mengklasifikasikan gambar.

Untuk mengakses dataset MNIST lakukan impor dan muat data dari MNIST langsung dari TensorFlow :

In [20]:
mnist = keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Dataset yang telah di muat akan mengembalikan empat nilai NumPy arrays:

* Array `train_images` dan `train_labels` merupakan data yang digunakan oleh model untuk mempelajari *pattern*
* Model diuji menggunakan *test set*, yaitu array `test_images`, dan `test_labels`.

Tiap gambar merupakan array berukuran 28x28, dengan nilai pixel yang berkisar antara 0 sampai dengan 255. Label dari data merupakan array bertipe bilangan integer, yang memiliki rentang nilai dari 0 sampai dengan 9. Nilai ini memiliki korespondensi dengan kelas angka (0-9).

Setiap gambar memiliki satu label saja. Nama kelas tidak tersedia dalam dataset, untuk itu pada kode berikut ditujukan untuk membuat nama kelas yang berguna ketika kita akan melakukan plotting dari gambar tersebut.

In [21]:
nama_kelas = ' '.join(map(str,[i for i in range(10)])).split(' ')

print(nama_kelas)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


## Eksplorasi Data

Lakukan eksplorasi dari format dataset sebelum menggunakan data tersebut untuk membangun model. Kode berikut menunjukan bahwa terdapat 60,000 gambar di dalam training set, setiap gambar berukuran 28 x 28 pixel.

In [22]:
train_images.shape

(60000, 28, 28)

Selain terdapat 60,000 gambar, terdapat juga 60,000 label di dalam training set:

In [23]:
len(train_labels)

60000

Setiap label adalah bilangan integer antara 0 dan 9:

In [24]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

Terdapat 10,000 gambar di dalam test set. Gambar ini juga direpresentasikan sebagai 28 x 28 pixel:

In [25]:
test_images.shape

(10000, 28, 28)

Dan test set mengandung 10,000 label gambar:

In [26]:
len(test_labels)

10000

## Pre-Processing Data

Data harus diolah terlebih dahulu sebelum digunakan untuk membangun model neural network. Jika Anda mengamati gambar pertama dalam training set, Anda akan melihat bahwa nilai pixel dari gambar memiliki rentang antara 0 sampai dengan 255: